# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load city data csv
cityDataDF = pd.read_csv("output_data/cities.csv")
cityDataDF.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,58.28,88,73,2.30,ZA,1635625527
1,1,esperance,-33.8667,121.9000,53.44,70,6,14.97,AU,1635625563
2,2,la vergne,36.0156,-86.5819,55.81,93,90,10.36,US,1635625564
3,3,bayangol,50.6977,103.4656,25.66,92,95,2.30,RU,1635625565
4,4,west wendover,40.7391,-114.0733,56.41,58,1,6.91,US,1635625566


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# heatmap of the humidity
locations = cityDataDF[["Lat","Lng"]]
humidity = cityDataDF["Humidity"]
fig = gmaps.figure()

# add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig
                        

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create dataFrame of cities with my ideal weather conditions 
# Max Temperature  between 70-80
# wind speed below 10
# cloudiness  between 20-30%

idealWeatherDF = cityDataDF.loc[
    (cityDataDF["Max Temp"] >= 70) & (cityDataDF["Max Temp"] <= 80) & \
    (cityDataDF["Wind Speed"] <= 10) & (cityDataDF["Cloudiness"] == 0)
].dropna()

#print(len(idealWeatherDF))

idealWeatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,127,hasilpur,29.7122,72.5553,70.99,22,0,2.24,PK,1635625658
158,158,karratha,-20.7377,116.8463,72.46,70,0,6.40,AU,1635625684
161,161,galeana,24.8333,-100.0667,72.01,27,0,1.03,MX,1635625686
180,180,sao jose da coroa grande,-8.8978,-35.1478,78.31,70,0,8.52,BR,1635625701
306,306,saint-pierre,-21.3393,55.4781,71.24,78,0,6.91,RE,1635625798


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df.
hotel_df = idealWeatherDF[["City","Lat","Lng", "Country"]]


In [7]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-7-9f48863c70ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Country,Hotel Name
127,hasilpur,29.7122,72.5553,PK,
158,karratha,-20.7377,116.8463,AU,
161,galeana,24.8333,-100.0667,MX,
180,sao jose da coroa grande,-8.8978,-35.1478,BR,
306,saint-pierre,-21.3393,55.4781,RE,
372,weatherford,32.7593,-97.7972,US,
387,santa maria del oro,25.9333,-105.3667,MX,
458,awbari,26.5921,12.7805,LY,
494,roebourne,-20.7833,117.1333,AU,
498,isangel,-19.5500,169.2667,VU,


In [17]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# loop through the dataframe using iterows()
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # use the lng & lat to set location value in the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # use google apis places base url seearch
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    
    name_address = requests.get(base_url, params=params).json()
    
    pprint(name_address)  #print ["results"][0]["name"]
    
    try:
        # populate the hotel name
        
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]['name']
    except:
        print(f"Error finding hotel in {row['City']}. Skipping...")
        
        pass
    
    hotel_df
    
    

{'html_attributions': [],
 'next_page_token': 'Aap_uECC1Yg1GyU_oP3D76lX8l33OGfh3vneWVjIFGooHpePTIiTqSS2TK7Q0ZAm5YlvOQjtKq_zSc6GunB-o1L8iEseYwSocTdXaqeDg0sb8h4rwK3gtpnYmeMUFulzv9UQk8IT6CdkrdUCcOq1OdCgkQeKnwdDJOb_kbDpVs00MqDDxj3bGe7Caxje1w4fR9WKuhWUeie-AUpPpC-y5KbDACB4s-QkOrk8nZyN-WIAeJdK3SuGue_yRUabRlj7CQHbbb5gAqxs5uppG-CxI9TUcMay1AjIE08IiWvkNiUGJI3f8JiFFWNIjHjdfSHoIILl8EsG_A-oJUIPcUjPkETPqDLx9CM6_Wvc-ltzVQhSbWQMEnCbsN4WuFJpSsSIxTHaeUkHqkliznUuUPyn03s2AEzYdErwMuAR_sR-Zg9lU0z0RRClWRRbzajL',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 29.7115825, 'lng': 72.5539094},
                           'viewport': {'northeast': {'lat': 29.71293143029149,
                                                      'lng': 72.5552559302915},
                                        'southwest': {'lat': 29.71023346970849,
                                                      'lng': 72.55255796970847}}},
              'icon': 'https://maps.gstatic.com/mapfiles

{'html_attributions': [],
 'next_page_token': 'Aap_uEDP2rYKSGHN1d5TvyBKG88mEm_XBfVtonsrT7Dft3HDB4_aSYsmIhwMlTBmlO6vm1ufysI-IwwMz_h_Eqaimcv-e8DRinm07c8vMEdi3vJZ_7eRXAMdmKLJ_bSN0UroqzgWJpZa513Ixk1pfiT4AnjWkbpwqV25gEGUiUaf8GKJYoi3gAFIe-70C1C_qWkIUR99JD-pQHqz4_cDqDksTZ1w_3vyOiMXIiaSW7thTGNFO8Ckbbko98Fx9KBTKIZwJLaFsHQZAJSRYhVex-WkNd93OPSYHAYFNVg6Pee6YBRdnxXSKIS-ktXqo-pg-QBkZfrTfp785j_yF-zEQzynWtgHViiim9syn2wSq30o6fbjVupyrK2YYxaKufFKjEhnOFswC07cdSI1iR4sJwfD66RkwWWDxDV0tAIjnIwIpjRrNNHsVZwWjOYD',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.7330245, 'lng': 116.8465499},
                           'viewport': {'northeast': {'lat': -20.7314352697085,
                                                      'lng': 116.8482617302915},
                                        'southwest': {'lat': -20.7341332302915,
                                                      'lng': 116.8455637697085}}},
              'icon': 'https://maps.gstatic.com/mapfi

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 24.8299411, 'lng': -100.0706387},
                           'viewport': {'northeast': {'lat': 24.8310256802915,
                                                      'lng': -100.0692159697085},
                                        'southwest': {'lat': 24.8283277197085,
                                                      'lng': -100.0719139302915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Mirador',
              'photos': [{'height': 4144,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/106170742827760958846">Jorge '
              

{'html_attributions': [],
 'next_page_token': 'Aap_uEBxtNP1ZznTJqK7Lo2y658I2_dIDOo1n7jDQika2fbCvLkgPdalVA3cl84jXM3PGvwffI7SXZc0KpeoqIF4mbeIKgbLyfeFD8Q-OeA_1kVqsDG0rv_aL6VHDRPGfDoBADaqlWqYqsuUK0uGx9pERqnKUSVueMqgqYNuUs5T-3fP7iifFktt05OxTM2Q9ZRVbVLLkiq2HQ6iddJwNmdC_dpjfF9ADcb0qwD-1wC9jN44oBirGiX7GXaQfmCCyfXeDLi0wsL481lYoQgfrVzjIVvGL_14q3gvHxsiNAmgmQ2TwVIHUWltkvWtNwO5S14UO99q00cfJbDS2dAiuhmNexeAOanE1SHr6-Aa09iGEbrlB0VVY-ifn_MycB8DGqGSUgWeUxhrrYgPz5bzUt5aRkQ8CYaN-cOSee5IyYAPEDVt6yr4uEAW0o8l',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -8.932708399999997,
                                        'lng': -35.16744190000001},
                           'viewport': {'northeast': {'lat': -8.931354419708498,
                                                      'lng': -35.1661069697085},
                                        'southwest': {'lat': -8.934052380291503,
                                                      'lng': -35.1688049302915}}

{'html_attributions': [],
 'next_page_token': 'Aap_uECjC9FgdbZSbP-yU4klPUSH64dcRoRvCI6KGUzbma7LZsvtLbDGNAqDMeWkfVWeYxH_bOGPAlVwMUfA5iVrE1wueG_tTjZ8Asg_MWUZbQCsnqRLIXToOQxgKbQoSEXBWq_x9NChgMmgKxNeNUbkFXbXYmscosDJ8uefVoVMtglL3QBoSuhoG1As3NkJ3AObQtRxxSDKs71tKtoWcRHUqonfDp6QGucWqRf2QHeBGwVbVx5-15aGT-lceT79Zv0vuxV1oLdpwSB4tJXAWDWHxp8_L_sqW0VETHgezpUrfH4FGfG7YPgG-LXMqv_0yb3l2cOn4Uk5BoPef6naGLrR5GQiHiO_KLZd2W1Sx69-qLFI4NAaFEwis5eEIAZytiA_NIVl-lVoW3nJcf8CkFihOLTuaSZi4iqUSyqYsB7SruXiitDKNYp2sRVb',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -21.3403342, 'lng': 55.4727123},
                           'viewport': {'northeast': {'lat': -21.3390070697085,
                                                      'lng': 55.47413358029149},
                                        'southwest': {'lat': -21.3417050302915,
                                                      'lng': 55.47143561970849}}},
              'icon': 'https://maps.gstatic.com/mapfil

{'html_attributions': [],
 'next_page_token': 'Aap_uEBotQNOgnH2_xGLGWLxpTt0IkifnGh4tK7hN6kPTBg9XlkkeC8HhXFCttThnAUtMzLR7mAUASH1htURP51cawOEPxsDMzpiGFPurA0QHAKnSrvC7efxrF0h-FRFpU-jMCD3cDqFqBxSb_FVNV1QNfz3sNNMf-l37V0W_WYSr2Q849Z6HceCkD5oRwpOFuzMqCf4es3bualH6fqMU_a7yXmAwiM-0eaLvvJ-ssEtKbp7qiDeTVI4ZQn_2ntqDpq6xXcJp4_6TTGpIpMNMKT6PeEMArdpjobCur836RKbU0MRypoRnPt20KhV27FdKNqnJQosm_H3_jWIXQ3f8kWYNfRVQPe3BWRaUOUPnky9iDC8EU-Cwz8DhWmTbRst0u5MK-4y3ThueDK38OTPc6CBfmlFdxujkXcgtItH7hAzt5l-OfrKaXubpos1',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.7268752,
                                        'lng': -97.79172040000002},
                           'viewport': {'northeast': {'lat': 32.72832098029151,
                                                      'lng': -97.7904124197085},
                                        'southwest': {'lat': 32.72562301970851,
                                                      'lng': -97.79311038029151}}},
      

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.9494518, 'lng': -105.363822},
                           'viewport': {'northeast': {'lat': 25.9507897302915,
                                                      'lng': -105.3625206697085},
                                        'southwest': {'lat': 25.94809176970849,
                                                      'lng': -105.3652186302915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Hotel Palacio Del Sol',
              'photos': [{'height': 1960,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/104897704721732072391">Banesa '

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.7544269, 'lng': 117.1457054},
                           'viewport': {'northeast': {'lat': -20.7530718197085,
                                                      'lng': 117.1472296302915},
                                        'southwest': {'lat': -20.7557697802915,
                                                      'lng': 117.1445316697085}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Latitude20 Roebourne Village',
              'place_id': 'ChIJK97NILx19isRHLj1hmjQIaw',
              'plus_code': {'compound_code': '64WW+67 Roebourne WA, Australia',
                            'global_code': '5PFV64WW+67'},
              'r

{'html_attributions': [],
 'next_page_token': 'Aap_uECjRYTf82OXZ5j-zw24gtdZn0pr6Ar_YawJzleEUc8DxqkY26Bi-3M0Aw4y6V9jcAve1rB_sAHr0MfayCYYU2tWP0qvsjhRsN62OsGFmy4FL3N1aGoCXtwLyaaolYwYdR0levaz3_YxPBqpctEPoWQY-4Td5etRiWYVqftnKBb9pyRfdY8x9Aqv-NZ4oQbQvCeWUGmp2FSzDSmwvo1Je-N1Z6ASHkwBY_sFVkOozjaO67QEldUTiAJIx0cnRK2oOHXeZuKXQs-2xBM-PB7EPUzbqraX-3YsGPa-C7c-xinl-0THU9D06orHNDQcqOmbf9tUkLuTFHKqpJ5UNfgmK1pkEsYOM9detieFwIUaSahxfl3rXHda9OMzMrAgOPv9NAfbAiizpaJm93CK7BDsC8CE31bVdLgthqapj7SmztsBoMICLztE07zH',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 26.4974601, 'lng': 76.7074425},
                           'viewport': {'northeast': {'lat': 26.4987977302915,
                                                      'lng': 76.7087586302915},
                                        'southwest': {'lat': 26.49609976970849,
                                                      'lng': 76.70606066970849}}},
              'icon': 'https://maps.gstatic.com/mapfiles/

In [18]:
# display dataFrame with hotels updated
hotel_df

,City,Lat,Lng,Country,Hotel Name
127,hasilpur,29.7122,72.5553,PK,"Masjid Noor, Tahir Colony, Purana Hasilpur"
158,karratha,-20.7377,116.8463,AU,ibis Styles Karratha
161,galeana,24.8333,-100.0667,MX,Mirador
180,sao jose da coroa grande,-8.8978,-35.1478,BR,Crôa Mares Hotel
306,saint-pierre,-21.3393,55.4781,RE,Lindsey Hôtel
372,weatherford,32.7593,-97.7972,US,"Candlewood Suites Weatherford, an IHG Hotel"
387,santa maria del oro,25.9333,-105.3667,MX,Hotel Palacio Del Sol
458,awbari,26.5921,12.7805,LY,فندق أوباري
494,roebourne,-20.7833,117.1333,AU,Latitude20 Roebourne Village
498,isangel,-19.5500,169.2667,VU,Tanna Lodge


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map


# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))